### from : rag_cot_peactice.md

## 환경 정보

In [24]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import PrivateAttr
from langchain.schema import BaseOutputParser
from langchain.embeddings import OpenAIEmbeddings
import numpy as np
import re

# .env 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")  # ex: "gpt-4o-mini"

# LLM 초기화: n개의 샘플 생성
llm = ChatOpenAI(
    model_name=OPENAI_LLM_MODEL,
    openai_api_key=OPENAI_API_KEY,
    temperature=0.7,
    # temperature=0.0,
    n=5
)

## AnswerParser

In [3]:
# Fake Retriever: Top-3 문서 반환
docs = {
    "doc1": "파리의 상징은 에펠탑이며, 1889년에 세워졌습니다.",
    "doc2": "파리는 세느강을 따라 발달한 도시로, 루브르 박물관이 유명합니다.",
    "doc3": "파리는 연간 약 2천만 명의 관광객이 방문하는 세계적 관광 도시입니다."
}
def fake_retriever(query: str, top_k: int=3):
    return [docs[f"doc{i}"] for i in range(1, top_k+1)]

class RobustSelfConsistencyParser(BaseOutputParser):
    threshold: float = 0.9
    _embeddings: OpenAIEmbeddings = PrivateAttr()

    def __init__(self, *, threshold: float = 0.9):
        super().__init__(threshold=threshold)
        self._embeddings = OpenAIEmbeddings()

    def parse(self, generations: list[str]) -> str:
        # 1) 우선 “최종 답변” 패턴으로 뽑아보기
        answers = []
        pattern = re.compile(r"(?:\d+\.\s*)?\**최종\s*답변\**[:\s]*(.+)", re.IGNORECASE|re.DOTALL)
        for text in generations:
            m = pattern.search(text)
            if m:
                answers.append(m.group(1).strip())
        # 2) 패턴 매칭이 하나도 안 되면, raw generation 전체를 답변 후보로 사용
        if not answers:
            answers = [text.strip() for text in generations]

        # 3) 의미 기반 클러스터링
        embs = np.array(self._embeddings.embed_documents(answers))
        rep_embs, counts, clusters = [], [], []
        for ans, emb in zip(answers, embs):
            for idx, rep in enumerate(rep_embs):
                sim = np.dot(emb, rep)/(np.linalg.norm(emb)*np.linalg.norm(rep))
                if sim >= self.threshold:
                    counts[idx] += 1
                    clusters[idx].append(ans)
                    break
            else:
                rep_embs.append(emb); counts.append(1); clusters.append([ans])

        # 4) 최빈 클러스터의 대표 답변 반환
        best = int(np.argmax(counts))
        return clusters[best][0]

### 난이도 하 (Easy)

In [4]:

query = "파리의 상징은 무엇인가요?"
retrieved = fake_retriever(query, top_k=2)
retrieved


['파리의 상징은 에펠탑이며, 1889년에 세워졌습니다.', '파리는 세느강을 따라 발달한 도시로, 루브르 박물관이 유명합니다.']

In [6]:
from langchain import PromptTemplate

# 1) Prompt → messages
prompt_easy = PromptTemplate(
    input_variables=["doc1","doc2","question"],
    template="""
=== 문서1 ===
{doc1}

=== 문서2 ===
{doc2}

질문: {question}

**최종 답변**:
"""
)

prompt_value = prompt_easy.format_prompt(
    doc1=retrieved[0],
    doc2=retrieved[1],
    question=query
)

messages = prompt_value.to_messages()  # ChatMessage 리스트



In [19]:
# 2) LLM으로 여러 답변 생성
response = llm.generate([messages])  
generations = response.generations[0]        # 5개의 Generation 객체
generations

[ChatGeneration(generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='파리의 에펠탑과 루브르 박물관을 포함한 역사와 문화를 체험하는 3일 여행 일정.', additional_kwargs={'refusal': None}, response_metadata={'finish_reason': 'stop', 'logprobs': None}, id='run--b9a4b2bc-f141-47f8-a8cb-31f7bb5ab03b-0', usage_metadata={'input_tokens': 188, 'output_tokens': 198, 'total_tokens': 386, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), text='파리의 에펠탑과 루브르 박물관을 포함한 역사와 문화를 체험하는 3일 여행 일정.'),
 ChatGeneration(generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='파리의 에펠탑과 루브르 박물관을 포함한 3일 여행 일정으로, 첫째 날은 에펠탑 방문, 둘째 날은 루브르 박물관 탐방, 셋째 날은 세느강 산책과 카페 문화 체험을 추천합니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'logprobs': None}, id='run--b9a4b2bc-f141-47f8-a8cb-31f7bb5ab03b-1', usage_metadata={'input_tokens': 188, 'output_tokens': 198, 'total_tokens': 386, 'input_token_details': {'audio': 0, '

In [23]:
for gen in generations:
    print(f"{gen.text}")
    

파리의 에펠탑과 루브르 박물관을 포함한 역사와 문화를 체험하는 3일 여행 일정.
파리의 에펠탑과 루브르 박물관을 포함한 3일 여행 일정으로, 첫째 날은 에펠탑 방문, 둘째 날은 루브르 박물관 탐방, 셋째 날은 세느강 산책과 카페 문화 체험을 추천합니다.
파리의 에펠탑과 루브르 박물관을 포함한 역사와 문화를 체험하는 3일 여행 일정.
파리의 에펠탑과 루브르 박물관을 포함한 역사와 문화를 체험하는 3일 여행 일정.
파리의 에펠탑과 루브르 박물관을 중심으로 세느강을 따라 3일 동안 문화와 역사를 탐방하는 일정.


In [13]:
# 3) 파서에 raw text 리스트 전달
raw_texts = [gen.text for gen in generations]
print("Easy raw texts:", raw_texts)
parser = RobustSelfConsistencyParser(threshold=0.85)
final_answer = parser.parse(raw_texts)

print("Easy 최종 답변:", final_answer)

Easy raw texts: ['파리의 에펠탑, 루브르 박물관, 세느강 유람선을 포함한 3일 여행 일정을 추천합니다.', '파리의 에펠탑과 루브르 박물관을 중심으로 한 3일 여행 일정 추천: 첫째 날 에펠탑 방문, 둘째 날 루브르 박물관 탐방, 셋째 날 세느강 주변 산책 및 관광.', '파리의 상징인 에펠탑을 방문하고, 루브르 박물관에서 세계적인 예술작품을 감상하며, 세느강 주변을 산책하는 3일 여행 일정을 추천합니다.', '파리의 에펠탑, 루브르 박물관, 세느강 유람선을 포함한 3일 여행 일정을 추천합니다.', '파리의 에펠탑, 루브르 박물관, 세느강 유람선을 즐기는 3일 여행 일정.']
Easy 최종 답변: 파리의 에펠탑, 루브르 박물관, 세느강 유람선을 포함한 3일 여행 일정을 추천합니다.


### 난이도 중 (Medium)

In [8]:
query = "파리의 주요 관광지 두 곳과 방문 시기를 알려주세요."
retrieved = fake_retriever(query, top_k=3)
retrieved


['파리의 상징은 에펠탑이며, 1889년에 세워졌습니다.',
 '파리는 세느강을 따라 발달한 도시로, 루브르 박물관이 유명합니다.',
 '파리는 연간 약 2천만 명의 관광객이 방문하는 세계적 관광 도시입니다.']

In [9]:
from langchain import PromptTemplate

# 1) Prompt → messages
prompt_med = PromptTemplate(
    input_variables=["doc1","doc2","doc3","question"],
    template="""
아래 문서를 참고하여 **한 번에 하나의 샘플**을 생성하세요.

=== 문서1 ===
{doc1}

=== 문서2 ===
{doc2}

=== 문서3 ===
{doc3}

질문: {question}

[지시사항]
- **최종 답변**: <답변 문장>
- 질문 문구를 반복하지 마세요.
- 부가 설명, 번호 없이 딱 한 줄로만 작성하세요.
"""
)

prompt_value = prompt_med.format_prompt(
    doc1=retrieved[0],
    doc2=retrieved[1],
    doc3=retrieved[2],
    question="파리의 주요 관광지 두 곳과 방문 시기를 알려주세요."
)

messages = prompt_value.to_messages()

# 2) LLM으로 여러 답변 생성
response = llm.generate([messages])
generations = response.generations[0]  # 5개의 Generation 객체

# 3) raw text 리스트 준비
raw_texts_med = [gen.text for gen in generations]

print(f"raw_texts_med: {raw_texts_med}")

# 4) 파싱
parser = RobustSelfConsistencyParser(threshold=0.9)
final_med = parser.parse(raw_texts_med)
print("Medium 최종 답변:", final_med)

raw_texts_med: ['파리의 주요 관광지는 에펠탑과 루브르 박물관이며, 방문하기 좋은 시기는 봄과 가을입니다.', '주요 관광지는 에펠탑과 루브르 박물관이며, 방문하기 좋은 시기는 봄과 가을입니다.', '파리의 주요 관광지는 에펠탑과 루브르 박물관이며, 연중 내내 방문할 수 있습니다.', '파리의 주요 관광지는 에펠탑과 루브르 박물관이며, 연중 언제든지 방문할 수 있습니다.', '에펠탑과 루브르 박물관은 파리의 주요 관광지로, 연중 내내 관광객이 방문합니다.']
Medium 최종 답변: 파리의 주요 관광지는 에펠탑과 루브르 박물관이며, 방문하기 좋은 시기는 봄과 가을입니다.


### 난이도 상 (Hard)

In [10]:
query = "파리의 역사, 관광지, 방문 시기를 종합해 3일 여행 일정을 추천해주세요."
retrieved = fake_retriever(query, top_k=3)
retrieved


['파리의 상징은 에펠탑이며, 1889년에 세워졌습니다.',
 '파리는 세느강을 따라 발달한 도시로, 루브르 박물관이 유명합니다.',
 '파리는 연간 약 2천만 명의 관광객이 방문하는 세계적 관광 도시입니다.']

In [11]:
# 1) Prompt → messages

prompt_hard_single = PromptTemplate(
    input_variables=["doc1","doc2","doc3","question"],
    template="""
아래 문서를 참고하여 **한 번에 하나의** 3일 일정 추천 샘플을 생성하세요.

=== 문서1 ===
{doc1}

=== 문서2 ===
{doc2}

=== 문서3 ===
{doc3}

질문: {question}

[지시사항]
- **최종 추천 일정**: <3일 일정 요약 한 줄>
- “### 일정 추천 샘플” 제목 없이, 한 줄짜리 요약만 작성하세요.
"""
)

prompt_value = prompt_hard_single.format_prompt(
    doc1=retrieved[0],
    doc2=retrieved[1],
    doc3=retrieved[2],
    question="파리의 역사, 관광지, 방문 시기를 종합해 3일 여행 일정을 추천해주세요."
)

messages = prompt_value.to_messages()

# 2) LLM으로 여러 답변 생성
response = llm.generate([messages])
generations = response.generations[0]

# 3) raw text 리스트 준비
raw_texts_hard = [gen.text for gen in generations]
print("Hard raw texts:", raw_texts_hard)

# 4) 파싱
final_hard = parser.parse(raw_texts_hard)
print("Hard 최종 답변:", final_hard)

Hard raw texts: ['파리의 에펠탑, 루브르 박물관, 세느강 유람선을 포함한 3일 여행 일정을 추천합니다.', '파리의 에펠탑, 루브르 박물관, 세느강 유람선을 즐기는 3일 여행 일정을 추천합니다.', '파리의 에펠탑, 루브르 박물관, 세느강 크루즈를 포함한 3일 일정으로 역사와 문화를 만끽하세요.', '파리의 에펠탑, 루브르 박물관, 세느강 유람선을 포함한 3일 여행 일정을 추천합니다.', '에펠탑 관람, 루브르 박물관 탐방, 세느강 유람선 투어로 구성된 파리 3일 여행 일정.']
Hard 최종 답변: 파리의 에펠탑, 루브르 박물관, 세느강 유람선을 포함한 3일 여행 일정을 추천합니다.
